# Part 1 - Business Problem

**Business Problem: AI-Assisted Diagnosis of Skin Cancer Using Deep Learning**

### Context

Skin cancer is one of the most common types of cancer, and early diagnosis is essential to increase the chances of a cure. Dermatoscopic examination requires specialized knowledge and can be subject to inter-observer variability. The use of artificial intelligence—especially convolutional neural networks (CNNs)—can **assist dermatologists** in detecting, classifying, and outlining suspicious skin lesions.

### Business Objective

**Develop an automated system based on Deep Learning that:**

1. **Classifies** images of skin lesions into different types (benign, melanoma, etc.).

2. **Automatically segments** the lesion area in the image, highlighting the suspicious region.

### Rationale / Added Value

* **Increased accuracy** in diagnosis, reducing false negatives and false positives.

* **Standardization** of medical reports, minimizing subjectivity.

* **Remote assistance** in regions with few dermatologists.

* **Faster** patient screening and care.

### Problem Formulation

**Problem 1: Classification**

* Given an image of a skin lesion, the model should assign it to one of the possible classes (melanoma, basal cell carcinoma, nevus, etc.).

* **Suggested metrics:** Accuracy, F1-score (to handle class imbalance).

**Problem 2: Segmentation**

* Given an image, the model should generate a **binary mask** indicating the exact area of the lesion.

* **Suggested metrics:** IoU (Intersection over Union), Dice Coefficient.

## Technical Approach

### Classification with CNNs

* Use standard architectures such as **ResNet, EfficientNet, VGG**, or custom models.

* Train the model to differentiate between types of skin lesions.

### Segmentation with U-Net

* U-Net is a CNN architecture designed for medical image segmentation.

* It enables the generation of a precise mask of the lesion area.

* This is essential for indicating the region of interest to clinicians, and can be combined with the classification output for greater reliability.

#### Example Pipeline

1. **Preprocessing:** Normalization, resizing, data augmentation.

2. **Training U-Net:** Image as input, mask as output.

3. **Evaluation:** Metrics such as Dice, IoU.

4. **Visualization:** Overlay the segmented mask on the original image.

## Example Problem Statement

> **"How can we use convolutional neural networks to automate the classification and segmentation of skin lesions, assisting dermatologists in the early diagnosis of skin cancer?"**

## Possible Extensions

* Automated medical reports (including lesion type and area).

* Embedded systems for clinics and doctor’s offices.

* Telemedicine triage platforms.

## Real-World Applications (References)

* [Google Health Skin AI](https://www.blog.google/technology/health/dermatology-ai-preview/)

* [Nature Medicine 2020 Study](https://www.nature.com/articles/s41591-020-0942-0) (same research group as your dataset)

## How to Use in Your Presentation/Proposal

You can start with a real-world context (case statistics, diagnostic challenges), present the dataset, explain the tasks, and propose a solution based on CNN/U-Net, demonstrating real impact in healthcare.

# **Part 2 - API Kaggle**

In [ ]:
import zipfile
from google.colab import files

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Downloading database from kaggle
!kaggle datasets download surajghuwalewala/ham1000-segmentation-and-classification

# Part 3 - Baixando dataset

In [ ]:
# Path to the zip file containing the Bitcoin tweets dataset
zf = "/content/ham1000-segmentation-and-classification.zip"

In [ ]:
# Target directory where the dataset will be extracted
target_dir = "/content/dataset/CNN/skin"

In [ ]:
# Open the zip file containing the dataset
zfile = zipfile.ZipFile(zf)

In [ ]:
# Extract all contents of the zip file to the target directory
zfile.extractall(target_dir)

# Part 4 - Importando as bibliotecas

In [ ]:
##

#
import os

#
import numpy as np

#
import seaborn as sns
import matplotlib.pyplot as plt

#
from PIL import Image

#
from glob import glob

#
from sklearn.model_selection import train_test_split

#
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout

#
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG16

#
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#
from tqdm import tqdm

In [ ]:
# Caminho dos dados
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
NUM_CLASSES = 7  # Troque pelo número de classes do seu problema

# **Part 5 - Database**

In [ ]:
# Lista dos caminhos completos das imagens e máscaras
image_paths = sorted(glob('/content/dataset/CNN/skin/images/*.jpg'))  # ou .png, ajuste se necessário
mask_paths = sorted(glob('/content/dataset/CNN/skin/masks/*.png'))    # geralmente máscara é PNG

print(f"Total de imagens: {len(image_paths)}")
print(f"Total de máscaras: {len(mask_paths)}")

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)


# **Part 6 - Data Generator**

In [ ]:
IMG_SIZE = (128, 128)

class DataGenerator(Sequence):
    def __init__(self, image_filenames, mask_filenames, batch_size=16, img_size=IMG_SIZE):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.batch_size = batch_size
        self.img_size = img_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, idx):
        batch_img = self.image_filenames[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_mask = self.mask_filenames[idx * self.batch_size : (idx + 1) * self.batch_size]

        X = np.zeros((len(batch_img), *self.img_size, 3), dtype=np.float32)
        y = np.zeros((len(batch_mask), *self.img_size, 1), dtype=np.float32)

        for i, (img_path, mask_path) in enumerate(zip(batch_img, batch_mask)):
            img = Image.open(img_path).resize(self.img_size)
            img = np.array(img) / 255.0
            if img.ndim == 2:
                img = np.stack([img]*3, axis=-1)
            X[i] = img

            mask = Image.open(mask_path).resize(self.img_size)
            mask = np.array(mask)
            if mask.ndim == 3:
                mask = mask[..., 0]  # se máscara for RGB, pega canal 0
            mask = np.expand_dims(mask, axis=-1) / 255.0
            y[i] = mask

        return X, y

# **Part 7 - Generators**

In [ ]:
#
BATCH_SIZE = 16

#
train_gen = DataGenerator(train_images, train_masks, batch_size=BATCH_SIZE)

#
val_gen = DataGenerator(val_images, val_masks, batch_size=BATCH_SIZE)

# **Part 8 - Rede Neural 1 U-Net**

In [ ]:
def build_unet(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)
    # Encoder
    c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D()(c1)
    c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
    p2 = MaxPooling2D()(c2)
    c3 = Conv2D(64, 3, activation='relu', padding='same')(p2)
    p3 = MaxPooling2D()(c3)
    # Bottleneck
    b = Conv2D(128, 3, activation='relu', padding='same')(p3)
    # Decoder
    u1 = Conv2DTranspose(64, 2, strides=2, padding='same')(b)
    u1 = concatenate([u1, c3])
    u2 = Conv2DTranspose(32, 2, strides=2, padding='same')(u1)
    u2 = concatenate([u2, c2])
    u3 = Conv2DTranspose(16, 2, strides=2, padding='same')(u2)
    u3 = concatenate([u3, c1])
    outputs = Conv2D(1, 1, activation='sigmoid')(u3)
    model = Model(inputs, outputs)
    return model

def iou_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.round(y_pred))  # arredonda p/ binário
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.round(y_pred))  # arredonda p/ binário
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

unet = build_unet()
unet.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy', iou_coef, dice_coef])

unet.summary()

In [ ]:
%%time

#
history = unet.fit(train_gen,
                   validation_data=val_gen,
                   epochs=5)

In [ ]:
# Função para plotar o desempenho
def plot_training_history(history):
    # Loss
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(False)

    # Accuracy (funciona para accuracy, IoU, Dice... se estiver no metrics do modelo)
    if 'accuracy' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Val Accuracy')
        plt.title('Accuracy over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(False)
    plt.tight_layout()
    plt.show()

# Chame a função após o treino
plot_training_history(history)

In [ ]:
val_loss, val_acc, val_iou, val_dice = unet.evaluate(val_gen)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['iou_coef'], label='Train IoU')
plt.plot(history.history['val_iou_coef'], label='Val IoU')
plt.title('IoU over Epochs')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history['dice_coef'], label='Train Dice')
plt.plot(history.history['val_dice_coef'], label='Val Dice')
plt.title('Dice Coefficient over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"Val Loss: {val_loss:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print(f"Val IoU: {val_iou:.4f}")
print(f"Val Dice: {val_dice:.4f}")

# **Part 9 - Avaliação métricas do CNN 1 U-Net**

In [ ]:
# Pegue um batch do seu generator (ou use X_val e y_val arrays)
X_val, y_val = val_gen[0]  # primeiro batch de validação
y_pred = unet.predict(X_val)

# Binarize as máscaras preditas
y_pred_bin = (y_pred > 0.5).astype(np.uint8)
y_true_bin = (y_val > 0.5).astype(np.uint8)


# Transforme as máscaras em vetores 1D (para cada pixel ser uma "amostra")
y_true_flat = y_true_bin.flatten()
y_pred_flat = y_pred_bin.flatten()

In [ ]:
# classification report U-Net
report = classification_report(y_true_flat, y_pred_flat, target_names=['Background', 'Lesion'])
print(report)

In [ ]:
# Gere as previsões e binarize
# X_val, y_val = val_gen[0]
# y_pred = unet.predict(X_val)

y_pred_bin = (y_pred > 0.5).astype(np.uint8)
y_true_bin = (y_val > 0.5).astype(np.uint8)

# Transforme em vetores 1D
y_true_flat = y_true_bin.flatten()
y_pred_flat = y_pred_bin.flatten()

In [ ]:
# Gere a matriz de confusão e visualize

cm = confusion_matrix(y_true_flat, y_pred_flat)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Background', 'Lesion'])

fig, ax = plt.subplots(figsize=(5, 5))
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix (per pixel)')
plt.show()

# **Part 10 - Previsão das imagens Rede Neural 1 - U-Net**

In [ ]:
# Pegando imagens e máscaras do val_gen (validação)
# Pegue um batch do generator de validação

# Pode mudar o índice para ver outros batches
X_val, y_val = val_gen[0]

In [ ]:
## Faça a predição
# Realize a predição (output entre 0 e 1)
y_pred = unet.predict(X_val)

In [ ]:
# Binarize a máscara predita
# Limiares > 0.5 viram "lesão"
y_pred_bin = (y_pred > 0.5).astype(np.uint8)

In [ ]:
## Visualize o resultado (imagem, ground truth, predição)

n = 5  # número de exemplos para mostrar
plt.figure(figsize=(12, n*3))

for i in range(n):
    plt.subplot(n, 3, i*3 + 1)
    plt.imshow(X_val[i])
    plt.title('Image')
    plt.axis('off')

    plt.subplot(n, 3, i*3 + 2)
    plt.imshow(y_val[i].squeeze(), cmap='gray')
    plt.title('Ground Truth')
    plt.axis('off')

    plt.subplot(n, 3, i*3 + 3)
    plt.imshow(y_pred_bin[i].squeeze(), cmap='gray')
    plt.title('Prediction')
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
## Fazer previsão em uma nova imagem (fora do dataset)

# Previsão em uma imagem específica (ex: img_path = "/content/my_image.jpg")

#
def predict_single_image(model, img_path, img_size=(128, 128)):
    img = Image.open(img_path).resize(img_size)
    img = np.array(img) / 255.0
    if img.ndim == 2:  # grayscale
        img = np.stack([img]*3, axis=-1)
    img = np.expand_dims(img, axis=0)  # (1, h, w, 3)
    pred = model.predict(img)
    mask_pred = (pred[0, :, :, 0] > 0.5).astype(np.uint8)
    return img[0], mask_pred

In [ ]:
##
# Exemplo de uso

#
img_path = "/content/dataset/CNN/skin/images/ISIC_0024306.jpg"

#
img, mask_pred = predict_single_image(unet, img_path)

In [ ]:
# Visualizando
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Image")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(mask_pred, cmap='gray')
plt.title("Predicted Mask")
plt.axis('off')
plt.show()

# **Part 11 - Salvando rede neural 1 U-Net**

In [ ]:
## Salvar o modelo completo (.h5 ou SavedModel)
# Forma clássica: salva tudo em único arquivo
unet.save('/content/unet_skin_segmentation.h5')

# Forma moderna: salva em formato "SavedModel" (pasta)
#unet.save('/content/unet_skin_segmentation_model')

In [ ]:
## Salvar apenas os pesos (caso queira restaurar numa arquitetura igual)

# Salva só os pesos
#unet.save_weights('/content/unet_weights.h5')

# **Part 12 - Rede Neural 2 - U-Net VGG16**

In [ ]:
def unet_with_vgg16_encoder(input_shape=(128, 128, 3), freeze_encoder=True):
    # Carrega VGG16 pré-treinado sem o topo
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    if freeze_encoder:
        vgg16.trainable = False

    # Encoder (skip connections)
    inputs = vgg16.input
    s1 = vgg16.get_layer('block1_conv2').output  # (128, 128, 64)
    s2 = vgg16.get_layer('block2_conv2').output  # (64, 64, 128)
    s3 = vgg16.get_layer('block3_conv3').output  # (32, 32, 256)
    s4 = vgg16.get_layer('block4_conv3').output  # (16, 16, 512)
    b = vgg16.get_layer('block5_conv3').output   # (8, 8, 512)

    # Decoder
    d1 = Conv2DTranspose(512, 2, strides=2, padding='same')(b)    # (16, 16, 512)
    d1 = concatenate([d1, s4])
    d1 = Conv2D(512, 3, activation='relu', padding='same')(d1)
    d1 = Conv2D(512, 3, activation='relu', padding='same')(d1)

    d2 = Conv2DTranspose(256, 2, strides=2, padding='same')(d1)   # (32, 32, 256)
    d2 = concatenate([d2, s3])
    d2 = Conv2D(256, 3, activation='relu', padding='same')(d2)
    d2 = Conv2D(256, 3, activation='relu', padding='same')(d2)

    d3 = Conv2DTranspose(128, 2, strides=2, padding='same')(d2)   # (64, 64, 128)
    d3 = concatenate([d3, s2])
    d3 = Conv2D(128, 3, activation='relu', padding='same')(d3)
    d3 = Conv2D(128, 3, activation='relu', padding='same')(d3)

    d4 = Conv2DTranspose(64, 2, strides=2, padding='same')(d3)    # (128, 128, 64)
    d4 = concatenate([d4, s1])
    d4 = Conv2D(64, 3, activation='relu', padding='same')(d4)
    d4 = Conv2D(64, 3, activation='relu', padding='same')(d4)

    # Saída (1 canal, para máscara binária)
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(d4)

    model2 = Model(inputs, outputs)
    return model2

# Criando o modelo
input_shape = (128, 128, 3)
model2 = unet_with_vgg16_encoder(input_shape=input_shape, freeze_encoder=True)

#
def iou_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.round(y_pred))
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

#
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.round(y_pred))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', iou_coef, dice_coef])

model2.summary()

In [ ]:
# Supondo que você já tenha train_gen e val_gen prontos
history2 = model2.fit(train_gen,
                      validation_data=val_gen,
                      epochs=5)

In [ ]:
print(history2.history.keys())


In [ ]:
def plot_training_history(history):
    metrics = [m for m in history.history.keys() if not m.startswith('val_')]
    n_metrics = len(metrics)

    plt.figure(figsize=(6 * n_metrics, 5))
    for i, metric in enumerate(metrics):
        plt.subplot(1, n_metrics, i+1)
        plt.plot(history.history[metric], label='Train ' + metric)
        if f'val_{metric}' in history.history:
            plt.plot(history.history[f'val_{metric}'], label='Val ' + metric)
        plt.title(metric.capitalize())
        plt.xlabel('Epoch')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(True)
    plt.tight_layout()
    plt.show()

# Chame assim:
plot_training_history(history2)

In [ ]:
# Apenas Loss
plt.plot(history2.history['loss'], label='Train Loss')
plt.plot(history2.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

# **Part 13 - Metricas e avaliação 2**

In [ ]:
# Pegue um batch do generator de validação
X_val, y_val = val_gen[0]   # Use outros índices para mais imagens

# Gere as previsões
y_pred = model2.predict(X_val)

# Binarize as máscaras preditas (> 0.5 vira 1)
y_pred_bin = (y_pred > 0.5).astype(np.uint8)
y_true_bin = (y_val > 0.5).astype(np.uint8)

In [ ]:
# Flatten para comparar pixel a pixel
y_true_flat = y_true_bin.flatten()
y_pred_flat = y_pred_bin.flatten()

In [ ]:
# Classification Report
print(classification_report(
    y_true_flat, y_pred_flat, target_names=['Background', 'Lesion']
))

In [ ]:
# Matriz de Confusão
cm = confusion_matrix(y_true_flat, y_pred_flat)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Background', 'Lesion'])

fig, ax = plt.subplots(figsize=(5, 5))
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix (per pixel)')
plt.show()

# **Part 14 - Previsãoes imagens**

In [ ]:
# Exemplo para várias batches (opcional)
y_true_all, y_pred_all = [], []
for i in range(len(val_gen)):

    X_batch, y_batch = val_gen[i]
    pred_batch = model2.predict(X_batch)
    y_true_all.append(y_batch)
    y_pred_all.append(pred_batch)

y_true_all = np.concatenate(y_true_all, axis=0)
y_pred_all = np.concatenate(y_pred_all, axis=0)
y_true_flat = (y_true_all > 0.5).astype(np.uint8).flatten()
y_pred_flat = (y_pred_all > 0.5).astype(np.uint8).flatten()

In [ ]:
# Visualização de Previsões
X_val, y_val = val_gen[0]
y_pred = model2.predict(X_val)
y_pred_bin = (y_pred > 0.5).astype(np.uint8)

In [ ]:
n = 5

plt.figure(figsize=(12, n*3))
for i in range(n):
    plt.subplot(n, 3, i*3 + 1)
    plt.imshow(X_val[i])
    plt.title('Image')
    plt.axis('off')

    plt.subplot(n, 3, i*3 + 2)
    plt.imshow(y_val[i].squeeze(), cmap='gray')
    plt.title('Ground Truth')
    plt.axis('off')

    plt.subplot(n, 3, i*3 + 3)
    plt.imshow(y_pred_bin[i].squeeze(), cmap='gray')
    plt.title('Prediction')
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def predict_single_image(model, img_path, img_size=(128,128)):
    # Carrega e prepara a imagem
    img = Image.open(img_path).resize(img_size)
    img_array = np.array(img) / 255.0
    if img_array.ndim == 2:  # Se for grayscale, converte para RGB
        img_array = np.stack([img_array]*3, axis=-1)
    img_input = np.expand_dims(img_array, axis=0)  # (1, h, w, 3)

    # Faz predição
    pred = model.predict(img_input)
    mask_pred = (pred[0, :, :, 0] > 0.5).astype(np.uint8)
    return img_array, mask_pred

In [ ]:
# Exemplo de uso

# Troque para o caminho da sua imagem!
img_path = '/content/dataset/CNN/skin/images/ISIC_0024311.jpg'

# Previsao modelo
img, mask_pred = predict_single_image(model2, img_path)

In [ ]:
# Visualização
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Image")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(mask_pred, cmap='gray')
plt.title("Predicted Mask")
plt.axis('off')
plt.show()

# **Part 15 - Salvando Rede Neural 2 VGG16**

In [ ]:
# Salvar em formato HDF5 (arquivo único)
model2.save('/content/unet_vgg16_skin.h5')

# Ou em formato SavedModel (pasta)
#model2.save('/content/unet_vgg16_skin_model')

#model2.save_weights('/content/unet_vgg16_skin_weights.h5')